In [2]:
from keras.layers import (
    LSTM,
    BatchNormalization,
    Conv1D,
    Conv1DTranspose,
    Dropout,
    Dense,
    GlobalAveragePooling1D,
    Input,
    ReLU,
    RepeatVector,
    TimeDistributed,
    MaxPool1D,
    UpSampling1D
)
from keras.models import Sequential, Model
import keras
from typing import List, Optional

In [3]:
def make_conv_ae_2(
    sequence_length: int,
    n_features: int = 1,
    kernel_size: int = 5,
    encoder_sizes: List[int] = [64, 32, 16, 8],
    encoder_strides: List[int] = [4, 4, 4, 4],
    encoder_dropout: Optional[float] = 0.2,
    decoder_sizes: Optional[List[int]] = None,
    decoder_strides: Optional[List[int]] = None,
    decoder_dropout: Optional[float] = None,
):
    """Create a Convolutional AutoEncoder anomaly detection model.

    Note: the returned model is uncompiled.
    Original model had a learning rate of 0.001: keras.optimizers.Adam(learning_rate=0.001)
    """
    if decoder_sizes is None:
        decoder_sizes = encoder_sizes[::-1]
    if decoder_strides is None:
        decoder_strides = encoder_strides[::-1]

    layers = [Input(shape=(sequence_length, n_features))]
    for i, (layer_size, stride) in enumerate(zip(encoder_sizes, encoder_strides)):
        encoder_layer = Conv1D(
            filters=layer_size,
            kernel_size=kernel_size,
            padding="same",
            strides=stride,
            activation="relu",
        )
        layers.append(encoder_layer)
        if encoder_dropout is not None and i != len(encoder_sizes) - 1:
            # don't add a dropout after the last encoder layer
            encoder_dropout_layer = Dropout(encoder_dropout)
            layers.append(encoder_dropout_layer)

    for i, (layer_size, stride) in enumerate(zip(decoder_sizes, decoder_strides)):
        decoder_layer = Conv1DTranspose(
            filters=layer_size,
            kernel_size=kernel_size,
            padding="same",
            strides=stride,
            activation="relu",
        )
        layers.append(decoder_layer)
        if decoder_dropout is not None and i != len(decoder_sizes) - 1:
            # don't add a dropout after the last decoder layer
            decoder_dropout_layer = Dropout(decoder_dropout)
            layers.append(decoder_dropout_layer)

    layers.append(
        Conv1DTranspose(filters=n_features, kernel_size=7, padding="same"),
    )
    return Sequential(layers)

In [18]:
make_conv_ae_2(sequence_length=2048,
               n_features=2,
               encoder_dropout = 0.2,
               encoder_sizes = [32, 16, 8],
               encoder_strides = [8, 8, 8],).summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_62 (Conv1D)           (None, 256, 32)           352       
_________________________________________________________________
dropout_52 (Dropout)         (None, 256, 32)           0         
_________________________________________________________________
conv1d_63 (Conv1D)           (None, 32, 16)            2576      
_________________________________________________________________
dropout_53 (Dropout)         (None, 32, 16)            0         
_________________________________________________________________
conv1d_64 (Conv1D)           (None, 4, 8)              648       
_________________________________________________________________
conv1d_transpose_75 (Conv1DT (None, 32, 8)             328       
_________________________________________________________________
conv1d_transpose_76 (Conv1DT (None, 256, 16)         

In [35]:
def make_conv_max_pool_ae(sequence_length: int, n_features: int = 1):
    """Create a Convolutional AutoEncoder anomaly detection model.

    Note: the returned model is uncompiled.
    Original model had a learning rate of 0.001: keras.optimizers.Adam(learning_rate=0.001)

    Args:
        sequence_length: number of timesteps in the timeseries.
        n_features: number of features.

    Examples:
        Create and compile a convolution based AutoEncoder:
        >>> model = make_conv_ae(2048)
        >>> model.compile(optimize="adam", loss="mse")
    """
    # TODO: play with kernel regularizers to reduce overfitting

    model = Sequential(
        [
            Input(shape=(sequence_length, n_features)),
            Conv1D(
                filters=32, kernel_size=15, padding="same", strides=8, activation="relu"
            ),
            # BatchNormalization(),
#             Dropout(rate=0.2),
            MaxPool1D(2, padding='same'),
            Conv1D(
                filters=16, kernel_size=15, padding="same", strides=8, activation="relu"
            ),
#             Dropout(rate=0.2),
#             MaxPool1D(4),
            MaxPool1D(2, padding='same'),
            Conv1D(
                filters=16, kernel_size=15, padding="same", strides=8, activation="relu"
            ),
#             Dropout(rate=0.2),
#             MaxPool1D(4),
#             MaxPool1D(2, padding='same'),
#             Conv1D(
#                 filters=8, kernel_size=15, padding="same", strides=4, activation="relu"
#             ),
            Conv1DTranspose(
                filters=8, kernel_size=15, padding="same", strides=4, activation="relu"
            ),
            UpSampling1D(2),
#             Dropout(rate=0.2),
            Conv1DTranspose(
                filters=16, kernel_size=15, padding="same", strides=4, activation="relu"
            ),
            UpSampling1D(2),
#             Dropout(rate=0.2),
            Conv1DTranspose(
                filters=32, kernel_size=15, padding="same", strides=4, activation="relu"
            ),
            UpSampling1D(2),
            # BatchNormalization(),
#             Dropout(rate=0.2),
            Conv1DTranspose(
                filters=64, kernel_size=15, padding="same", strides=4, activation="relu"
            ),
            Conv1DTranspose(filters=2, kernel_size=7, padding="same"),
        ]
    )
    return model
model = make_conv_max_pool_ae(2048, 1)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
model.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_62 (Conv1D)           (None, 256, 32)           512       
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 128, 32)           0         
_________________________________________________________________
conv1d_63 (Conv1D)           (None, 16, 16)            7696      
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 8, 16)             0         
_________________________________________________________________
conv1d_64 (Conv1D)           (None, 1, 16)             3856      
_________________________________________________________________
conv1d_transpose_82 (Conv1DT (None, 4, 8)              1928      
_________________________________________________________________
up_sampling1d_24 (UpSampling (None, 8, 8)            

In [52]:
def create_model(sequence_length, n_features):
    model = Sequential(
        [
            Input(shape=(sequence_length, n_features)),
            Conv1D(
                filters=32, kernel_size=15, padding="same", strides=8, activation="relu"
            ),
            # BatchNormalization(),
#             Dropout(rate=0.2),
#             Conv1D(
#                 filters=32, kernel_size=3, padding="same", strides=4, activation="relu"
#             ),
            Dropout(rate=0.2),
            Conv1D(
                filters=16, kernel_size=15, padding="same", strides=8, activation="relu"
            ),
#             Dropout(rate=0.2),
#             Conv1D(
#                 filters=8, kernel_size=3, padding="same", strides=4, activation="relu"
#             ),
            Dropout(rate=0.2),
             Conv1D(
                filters=8, kernel_size=3, padding="same", strides=8, activation="relu"
            ),
            Conv1DTranspose(
                filters=8, kernel_size=3, padding="same", strides=4, activation="relu"
            ),
#             Conv1DTranspose(
#                 filters=8, kernel_size=3, padding="same", strides=4, activation="relu"
#             ),
#             Dropout(rate=0.2),
            Conv1DTranspose(
                filters=32, kernel_size=3, padding="same", strides=4, activation="relu"
            ),
#             Dropout(rate=0.2),
#             Conv1DTranspose(
#                 filters=32, kernel_size=3, padding="same", strides=4, activation="relu"
#             ),
            # BatchNormalization(),
            Dropout(rate=0.2),
            Conv1DTranspose(
                filters=64, kernel_size=3, padding="same", strides=4, activation="relu"
            ),
            Conv1DTranspose(filters=2, kernel_size=3, padding="same"),
        ]
    )
    return model
create_model(2048, 2).summary()

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_113 (Conv1D)          (None, 256, 32)           992       
_________________________________________________________________
dropout_78 (Dropout)         (None, 256, 32)           0         
_________________________________________________________________
conv1d_114 (Conv1D)          (None, 32, 16)            7696      
_________________________________________________________________
dropout_79 (Dropout)         (None, 32, 16)            0         
_________________________________________________________________
conv1d_115 (Conv1D)          (None, 4, 8)              392       
_________________________________________________________________
conv1d_transpose_151 (Conv1D (None, 16, 4)             100       
_________________________________________________________________
conv1d_transpose_152 (Conv1D (None, 64, 32)          

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_56 (Conv1D)           (None, 2048, 64)          256       
_________________________________________________________________
dropout_84 (Dropout)         (None, 2048, 64)          0         
_________________________________________________________________
conv1d_57 (Conv1D)           (None, 2048, 32)          6176      
_________________________________________________________________
dropout_85 (Dropout)         (None, 2048, 32)          0         
_________________________________________________________________
conv1d_58 (Conv1D)           (None, 2048, 16)          1552      
_________________________________________________________________
dropout_86 (Dropout)         (None, 2048, 16)          0         
_________________________________________________________________
conv1d_59 (Conv1D)           (None, 2048, 8)         